In [2]:
# %matplotlib widget
# %load_ext autoreload
# %autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from decouple import Config, RepositoryEnv

DOTENV_FILE = os.path.dirname(os.path.dirname(os.path.abspath(''))) + "/local.env"
env_config = Config(RepositoryEnv(DOTENV_FILE))

In [3]:
DOTENV_FILE

'/home/ipausers/bisot/pycode/MscThesis/amftrack/local.env'

In [4]:
directory = '/mnt/sun-temp/TEMP/PRINCE_syncing/'
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']=='860']

In [25]:
p = dropbox.common.PathRoot.namespace_id("<TEAM_FOLDER_ID>")
dbx = dropbox.Dropbox(API, timeout=900).with_path_root(p)

ValueError: path_root must be an instance of PathRoot

In [18]:
dbx = dropbox.Dropbox(app_key = "nr0yly4qchgn8mb",
            app_secret = "5txb65rtycw7euv", oauth2_refresh_token = 'dXL7p4GVDQUAAAAAAAAA4Kb15YrA5LLXz1AMr7Xhng4')

In [20]:
# import requests

# app_key = "nr0yly4qchgn8mb"
# app_secret = "5txb65rtycw7euv"

# # build the authorization URL:
# authorization_url = "https://www.dropbox.com/oauth2/authorize?client_id=%s&response_type=code" % app_key

# # send the user to the authorization URL:
# print ('Go to the following URL and allow access:')
# print(authorization_url)

# get the authorization code from the user:
authorization_code = "dXL7p4GVDQUAAAAAAAAA4Kb15YrA5LLXz1AMr7Xhng4"

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "code": authorization_code,
    "grant_type": "authorization_code",
    "client_id": app_key,
    "client_secret": app_secret
}
r = requests.post(token_url, data=params)
print(r.text)

{"access_token": "sl.BGBtXKEVO_cf1XyuOhsv3zj_MbJKkJ0D2T7zz70ZztCNSoak6lqxOxNs2nvf8hy2X04uuuXZXipV9bb4MqDrY263hZZkOgKag4lP1dxiAcCXYQjaelooV7eAHvPxWFz2-uQGIYM", "token_type": "bearer", "expires_in": 14400, "scope": "account_info.read files.content.read files.content.write files.metadata.read files.metadata.write", "uid": "3881053136", "account_id": "dbid:AABIDXJq597-uS2e5SgTFZ0vARb3NPh-G8s"}


In [25]:
oauth_result.refresh_token

'BKO7AIMPASAAAAAAAAAAAScV2k7pBzWireMMbryRw1cKlnldbp2I0divkm3qgdrm'

In [ ]:
import dropbox
from dropbox import DropboxOAuth2FlowNoRedirect

'''
Populate your app key in order to run this locally
'''
APP_KEY = "nr0yly4qchgn8mb"

auth_flow = DropboxOAuth2FlowNoRedirect(APP_KEY, use_pkce=True, token_access_type='offline')

authorize_url = auth_flow.start()
print("1. Go to: " + authorize_url)
print("2. Click \"Allow\" (you might have to log in first).")
print("3. Copy the authorization code.")
auth_code = input("Enter the authorization code here: ").strip()

try:
    oauth_result = auth_flow.finish(auth_code)
except Exception as e:
    print('Error: %s' % (e,))
    exit(1)

with dropbox.Dropbox(oauth2_refresh_token=oauth_result.refresh_token, app_key=APP_KEY) as dbx:
    dbx.users_get_current_account()
    print("Successfully set up client!")

1. Go to: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=nr0yly4qchgn8mb&token_access_type=offline&code_challenge=XLdM8ez2fjanTbpGoCgxN85mFW6hsoO9wUJ1NZ4-Mu8&code_challenge_method=S256
2. Click "Allow" (you might have to log in first).
3. Copy the authorization code.


In [22]:
oauth_result

OAuth2FlowNoRedirectResult(sl.BGCoDYjR-CfbD5rTinR4C70ZFeoVvfDA5PsIIyTh4o3FsUGrZrwYmaLmwClzff9gsafp7fzbno5O90MEm3nsVb2tdkPpQ94rYgEfQ2G0sCi-EXFiJI9qutCwUYc7z7oAmxVbjbM, dbid:AABIDXJq597-uS2e5SgTFZ0vARb3NPh-G8s, 3881053136, BKO7AIMPASAAAAAAAAAAAScV2k7pBzWireMMbryRw1cKlnldbp2I0divkm3qgdrm, 2022-04-19 17:37:56.702663, account_info.read files.content.read files.content.write files.metadata.read files.metadata.write)

In [7]:
env_config = Config(RepositoryEnv(DOTENV_FILE))
dir_drop = '/test'
# API = env_config.get("API_KEY")
API = "sl.BFxtWRWbjzZfa0GiP8qvfsmDLs-SsK34r2G09NRfOUTvz3jDk_rwFpfD6Ig7Q9SbYcPOkMN44jErViOYV0jHCMUIFGQGOEVeyIKSRb8K_qHguDYOpyS2G2USOw1018N_Chu4vfk"
temp_path = env_config.get("TEMP_PATH")


In [19]:
with open("sync_prince.py", "rb") as f:
    dbx.files_upload(f.read(), '/test/test.py', mode=dropbox.files.WriteMode.overwrite)

AuthError: AuthError('6dd9819542984703815e71b9c9c85da6', AuthError('invalid_access_token', None))

In [9]:
upload(API,"sync_prince.py",'/test/test.py')

Unable to refresh access token without                 refresh token and app key


AuthError: AuthError('062ddc4437b749c7807831d37da078ea', AuthError('expired_access_token', None))

In [22]:
run_info = folders.copy()
folder_list = list(run_info['folder'])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info['unique_id'] = run_info['Plate'].astype(str) + "_" + run_info['CrossDate'].astype(str)
        line = run_info.loc[run_info['folder']==directory_name]
        id_unique = line['unique_id'].iloc[0]

        path_snap=directory+directory_name
        path_info =f'{temp_path}/{directory_name}_info.json'
        
        for subfolder in ["Img","Analysis"]:
            path_zip =f'{temp_path}/{directory_name}_{subfolder}.zip'
            line.to_json(path_info)
            zip_file(os.path.join(path_snap,subfolder),path_zip)
            upload(API,path_zip,f'/{dir_drop}/{id_unique}/{directory_name}/{subfolder}.zip',chunk_size=256 * 1024 * 1024)
        upload(API,path_info,f'/{dir_drop}/{id_unique}/{directory_name}_info.json',chunk_size=256 * 1024 * 1024)
        os.remove(path_info) 
        os.remove(path_zip)
        pbar.update(1)

transferred:   0%|          | 0/115 [00:00<?, ?it/s]

zipping:   0%|          | 0/128 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/856440590 [00:00<?, ?it/s]

error


KeyboardInterrupt: 

In [20]:
os.path.join(path_snap,subfolder)

'/mnt/sun-temp/TEMP/PRINCE_syncing/20220321_0838_Plate11/Img'